# 5 - Uploading to S.A.M. bucket

## 1. Load .env variables



In [1]:
# Replace with your actual bucket name
bucket_name="sam_mv2"

In [8]:
%cd ../..

/home/arthurcornelio/code/arthurcornelio88/stable-audio-tools-sam/sam_files


In [4]:
%%bash -s "$bucket_name"

# Use the passed variable directly
echo "bucket_name=$1" >> "notebooks/dataset for fine-tuning/.env"

In [4]:
# load .env variables
from dotenv import load_dotenv
import os

load_dotenv()

GITHUB_PROFILE_NAME = os.getenv('GITHUB_PROFILE_NAME')
genre = os.getenv('genre')
file_count = os.getenv('file_count')
final_folder_name = os.getenv('final_folder_name')
bucket_name=os.getenv('bucket_name')
timestamp=os.getenv('timestamp')


## 2. Move manually your .JSON credential
  - In SAM Google Drive, go in data/sam-gcp
  - Put your json-credential.json in /sam_files
  - Open the .env for sam_files (sam_files/notebooks/dataset for fine-tuning/.env)
  - insert this line :
    - `gcp_api_json_path={json-credential.json}` (it's already in sam_files)

## 3. Uploading to Google Cloud Bucket

### 3.1. You need to be in :
  - `../stable-audio-tools-sam/sam_files`

### 3.2. Copying folders to bucket

- Run the cells above

In [ ]:
%%bash

# List of source folders you want to upload
source_folders=("audio_files/by_genre/${final_folder_name}")

# Upload each folder to the bucket
for source_folder in "${source_folders[@]}"
do
    gsutil -m cp -r -L "upload_log_${timestamp}.txt" "$source_folder"/* "gs://${bucket_name}/${final_folder_name}/"
done

### 3.3. Verify if uploading operation is successful

In [ ]:
# verify upload to GCP Bucket
from google.cloud import storage
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path='notebooks/dataset for fine-tuning/.env')

# Access the variables from the .env file
bucket_name = os.getenv('bucket_name')
folder_name = os.getenv('final_folder_name')

# Get the path to your service account credentials JSON file from the environment variable
credentials_path = os.getenv('gcp_api_json_path')

# Explicitly create the storage client using the service account credentials
storage_client = storage.Client.from_service_account_json(credentials_path)

def verify_upload(bucket_name, folder_name):
    """Verifies if the specified folder exists in the given GCS bucket."""

    bucket = storage_client.bucket(bucket_name)

    # Check if the main folder exists
    blobs = list(bucket.list_blobs(prefix=folder_name + '/'))
    if not blobs:
        print(f"Upload failed or incomplete. Main folder '{folder_name}' not found in bucket '{bucket_name}'.")
    else:
        print(f"Upload successful! Folder '{folder_name}' found in bucket '{bucket_name}'.")

# Call the function to verify only the folder_name
verify_upload(bucket_name, folder_name)

## 4. Delete final operation folders 

In [11]:
%%bash

# Delete the folders and their contents
rm -rf audio_files/final_backup audio_files/by_genre json/*

# Delete the folders and their contents
rm -rf "notebooks/dataset for fine-tuning/myenv"

rm -rf dataframes/* dataframes_${TIMESTAMP_DF}/* audio_files/final_backup/* audio_files/by_genre/*  json/*

# All done, bravo ! (by Arthur Cornélio, 12th August 2024)